In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Power consumption and Dark Silicon

## How fast are my CPUs and GPUs?

In [2]:
! lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   24
  On-line CPU(s) list:    0-23
Vendor ID:                GenuineIntel
  Model name:             13th Gen Intel(R) Core(TM) i7-13700
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   16
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   40%
    CPU max MHz:          5200.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4224.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc c

In [2]:
! cat /proc/cpuinfo |grep MHz

cpu MHz		: 2983.423
cpu MHz		: 2985.051
cpu MHz		: 2900.000
cpu MHz		: 2890.937
cpu MHz		: 2905.954
cpu MHz		: 2893.933
cpu MHz		: 800.000
cpu MHz		: 2845.663
cpu MHz		: 2199.575
cpu MHz		: 2900.000
cpu MHz		: 3209.328
cpu MHz		: 3170.649
cpu MHz		: 2896.102
cpu MHz		: 2895.966
cpu MHz		: 3119.893
cpu MHz		: 2975.159
cpu MHz		: 2550.740
cpu MHz		: 2595.774
cpu MHz		: 2670.763
cpu MHz		: 2620.689
cpu MHz		: 2620.637
cpu MHz		: 2627.226
cpu MHz		: 2661.067
cpu MHz		: 2600.827


In [3]:
! ssh htseng@eevee "cat /proc/cpuinfo"

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) Silver 4108 CPU @ 1.80GHz
stepping	: 4
microcode	: 0x2007006
cpu MHz		: 800.494
cache size	: 11264 KB
physical id	: 0
siblings	: 16
core id		: 0
cpu cores	: 8
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 22
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid dca sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb cat_l3 cdp_l3 invpcid_single pti intel_ppin ssbd mba ibrs ibpb stibp tpr_shadow vnmi flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm cqm mpx rdt_a avx51

In [16]:
! ssh htseng@eevee "cat /proc/cpuinfo|grep MHz"

cpu MHz		: 812.957
cpu MHz		: 801.587
cpu MHz		: 826.311
cpu MHz		: 800.107
cpu MHz		: 800.734
cpu MHz		: 800.028
cpu MHz		: 800.042
cpu MHz		: 909.626
cpu MHz		: 800.284
cpu MHz		: 800.262
cpu MHz		: 800.195
cpu MHz		: 805.568
cpu MHz		: 800.551
cpu MHz		: 800.049
cpu MHz		: 800.389
cpu MHz		: 800.090
cpu MHz		: 801.075
cpu MHz		: 881.479
cpu MHz		: 872.765
cpu MHz		: 800.041
cpu MHz		: 800.698
cpu MHz		: 800.026
cpu MHz		: 800.127
cpu MHz		: 1543.160
cpu MHz		: 800.226
cpu MHz		: 800.214
cpu MHz		: 800.349
cpu MHz		: 811.241
cpu MHz		: 800.068
cpu MHz		: 800.556
cpu MHz		: 800.065
cpu MHz		: 800.448


In [4]:
! ssh htseng@unown "nvidia-smi -a"


==============NVSMI LOG==============

Timestamp                                 : Thu Jun  6 10:19:32 2024
Driver Version                            : 535.54.03
CUDA Version                              : 12.2

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Product Name                          : NVIDIA GeForce RTX 4090
    Product Brand                         : GeForce
    Product Architecture                  : Ada Lovelace
    Display Mode                          : Disabled
    Display Active                        : Disabled
    Persistence Mode                      : Enabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial N

In [6]:
! ssh htseng@eevee "cd /nfshome/htseng/courses/CS203/demo/memory;  sudo likwid-perfctr -g ENERGY matrix_mul/blockmm 1024 256" | grep " |"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|         Event         | Counter |  Core 0 | Core 1 | Core 2 | Core 3 | Core 4 | Core 5 | Core 6 | Core 7 |        Core 8       | Core 9 | Core 10 | Core 11 | Core 12 | Core 13 | Core 14 | Core 15 | Core 16 | Core 17 | Core 18 | Core 19 | Core 20 | Core 21 | Core 22 | Core 23 | Core 24 | Core 25 | Core 26 | Core 27 | Core 28 | Core 29 | Core 30 | Core 31 |
|   INSTR_RETIRED_ANY   |  FIXC0  |       0 |      0 |      0 |  55705 | 142028 |      0 |  18985 | 171420 | 1.4074933622496e+14 |  41264 |       0 |  912477 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |   29221 |       0 |       0 |       0 |       0 |   64868 |
| CPU_CLK_UNHALTED_CORE |  FIXC1  |       0 |      0 |      0 |  81973 | 241066 |      0 |  80463 | 258717 | 1.4074263520275e+14 |  65386 | 

In [10]:
! ssh htseng@eevee "/nfshome/htseng/courses/CS203/demo/power/lowerMaxFreq.sh; cd /nfshome/htseng/courses/CS203/demo/memory;  sudo likwid-perfctr -g ENERGY matrix_mul/blockmm 1024 256" | grep " |"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|         Event         | Counter |  Core 0  | Core 1 | Core 2 | Core 3 | Core 4 | Core 5 | Core 6 | Core 7 |        Core 8       | Core 9 | Core 10 | Core 11 | Core 12 | Core 13 | Core 14 | Core 15 | Core 16 | Core 17 | Core 18 | Core 19 | Core 20 | Core 21 | Core 22 | Core 23 | Core 24 | Core 25 | Core 26 | Core 27 | Core 28 | Core 29 | Core 30 | Core 31 |
|   INSTR_RETIRED_ANY   |  FIXC0  |     8040 |      0 |      0 |   2279 |   2097 |  51192 |      0 |   4795 | 1.4074934310464e+14 |  41265 |       0 |  908856 |       0 |       0 |       0 |       0 |       0 |  141778 |       0 |  404680 |       0 |  310691 |       0 |   71739 |       0 |       0 |   29221 |       0 |       0 |  156400 |       0 |   65751 |
| CPU_CLK_UNHALTED_CORE |  FIXC1  |    15341 |      0 |      0 |  22090 |  13291 | 228745 |      0 |  24260 | 1.4074195855436e+14 |  67227

In [9]:
! ssh htseng@eevee "/nfshome/htseng/courses/CS203/demo/power/restoreMaxFreq.sh; cd /nfshome/htseng/courses/CS203/demo/memory;  sudo likwid-perfctr -g ENERGY matrix_mul/blockmm 1024 256" | grep " |"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|         Event         | Counter |  Core 0 | Core 1 | Core 2 | Core 3 | Core 4 | Core 5 | Core 6 | Core 7 |        Core 8       | Core 9 | Core 10 | Core 11 | Core 12 | Core 13 | Core 14 | Core 15 | Core 16 | Core 17 | Core 18 | Core 19 | Core 20 | Core 21 | Core 22 | Core 23 | Core 24 | Core 25 | Core 26 | Core 27 | Core 28 | Core 29 | Core 30 | Core 31 |
|   INSTR_RETIRED_ANY   |  FIXC0  |       0 |      0 |      0 |      0 |      0 |  15176 | 107571 |  63726 | 1.4074933620866e+14 |  41503 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |     226 |       0 |       0 |       0 |       0 |       0 |       0 |       0 |   29221 |       0 |       0 |       0 |       0 |   64847 |
| CPU_CLK_UNHALTED_CORE |  FIXC1  |       0 |      0 |      0 |      0 |      0 |  63743 | 209930 |  91618 | 1.4074263177762e+14 |  72490 | 